In [91]:
import json
import psycopg
from pgvector.psycopg import register_vector
import re
import os

conn = psycopg.connect(
    dbname="fpkg",
    user=os.environ.get("PGUSER"),
    password=os.environ.get("PGPASSWORD"),
    host="localhost",
    port="5431"
)

cursor = conn.cursor()

cursor.execute("LOAD 'age';")
cursor.execute("SET search_path = ag_catalog, \"$user\", public;")

conn.commit()

In [37]:
# Finding the list of nodes to embed
query = f"""
    SELECT name, relation
    FROM ag_catalog.ag_label
    WHERE kind = 'v' AND name != '_ag_label_vertex'
    AND graph = (SELECT graphid FROM ag_catalog.ag_graph WHERE name = 'fcsv')
"""

try:
    cursor.execute(query)
    results = cursor.fetchall()
    print(len(results))
    for res in results[:200]:
        # res = re.sub(r'::\w+$', '', res[0])
        print(res)

    vertex_labels = results
except Exception as e:
    print(e)
    conn.rollback()

8
('Drug', 'fcsv."Drug"')
('ActiveIngredient', 'fcsv."ActiveIngredient"')
('Excipient', 'fcsv."Excipient"')
('GenericGroup', 'fcsv."GenericGroup"')
('LegalSubstanceList', 'fcsv."LegalSubstanceList"')
('Indication', 'fcsv."Indication"')
('Contraindication', 'fcsv."Contraindication"')
('ROA', 'fcsv."ROA"')


In [ ]:
all_nodes = []

# Query each vertex label table
for label_name, table_relation in vertex_labels:
    try:
        # Now that we retrieved the label names, we can embed them
        cursor.execute(f"""
            SELECT * FROM cypher('fcsv', $$
                MATCH (v:{label_name})
                RETURN v.id, v.name, labels(v)
            $$) AS (v_id agtype, v_name agtype, v_labels agtype);
        """)
        label_nodes = cursor.fetchall()
        all_nodes.extend(label_nodes)
        print(f"Found {len(label_nodes)} nodes in {label_name}")
        
    except Exception as label_error:
        print(f"  Error querying {label_name}: {str(label_error)}")
        continue



[('"1"', '"A 313 50000UI CAPSULE"', '["Drug"]'), ('"2"', '"A 313 POMMADE TB 50G"', '["Drug"]'), ('"3"', '"ABACAVIR ARW 300MG CPR"', '["Drug"]'), ('"4"', '"ABACAVIR LAMIVUDIN ACC 600/300MG CPR"', '["Drug"]'), ('"5"', '"ABACAVIR LAMIVUDIN ARW 600/300MG CPR"', '["Drug"]'), ('"6"', '"ABACAVIR LAMIVUDIN BGA 600/300MG CPR"', '["Drug"]'), ('"7"', '"ABACAVIR LAMIVUDIN EG 600/300MG CPR"', '["Drug"]'), ('"8"', '"ABACAVIR LAMIVUDIN SDZ 600/300MG CPR"', '["Drug"]'), ('"9"', '"ABACAVIR LAMIVUDIN TVC 600/300MG CPR"', '["Drug"]'), ('"10"', '"ABACAVIR LAMIVUDIN VIA 600/300MG CPR"', '["Drug"]'), ('"11"', '"ABACAVIR LAMIVUDIN ZEN 600/300MG CPR"', '["Drug"]'), ('"12"', '"ABACAVIR LAMIVUDINE ZIDOVUDI VIA CPR"', '["Drug"]'), ('"13"', '"ABACAVIR SDZ 300MG CPR"', '["Drug"]'), ('"14"', '"ABACAVIR VIA 300MG CPR"', '["Drug"]'), ('"15"', '"ABASAGLAR 100U/ML INJ CART 3ML"', '["Drug"]'), ('"16"', '"ABASAGLAR KPEN 100U/ML STYLO 3ML"', '["Drug"]'), ('"17"', '"ABECMA 260-500MN CELL DISP INJ 50ML"', '["Drug"]'), ('"18

In [80]:
# Generate table to store embeddings
query = """
    SELECT * FROM document_vectors.columns();
"""

try:
    cursor.execute(query)
    results = cursor.fetchall()
    print(len(results))
    for res in results[:200]:
        # res = re.sub(r'::\w+$', '', res[0])
        print(res)

    vertex_labels = results
except Exception as e:
    print(e)
    conn.rollback()

schema "document_vectors" does not exist
LINE 2:     SELECT * FROM document_vectors.columns();
                          ^


In [93]:
print("\nAdding vector embeddings...")

# Create vector extension and table in a separate transaction
try:
    cursor.execute('CREATE EXTENSION IF NOT EXISTS vector')
    conn.commit()
    print("Vector extension created/verified")
    
    # Register vector extension
    register_vector(conn)
    
    # Check if table exists first
    cursor.execute("""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_name = 'document_vectors'
        );
    """)
    table_exists = cursor.fetchone()[0]
    
    if not table_exists:
        print("Creating document_vectors table...")
        cursor.execute("""
            CREATE TABLE document_vectors (
                id TEXT PRIMARY KEY,
                node_name TEXT,
                node_label TEXT,
                embedding vector(1536)
            );
        """)
        conn.commit()
        print("Document vectors table created successfully")
    else:
        cursor.execute("DROP TABLE document_vectors CASCADE;")
        conn.commit()
        print("Document vectors table deleted successfully")
        
    # Verify the table structure
    cursor.execute("""
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = 'document_vectors'
        ORDER BY ordinal_position;
    """)
    columns = cursor.fetchall()
    print(f"Table columns: {columns}")
    
except Exception as e:
    print(f"Error setting up table: {str(e)}")
    conn.rollback()


Adding vector embeddings...
Vector extension created/verified
Creating document_vectors table...
Document vectors table created successfully
Table columns: [('id', 'text'), ('node_name', 'text'), ('node_label', 'text'), ('embedding', 'USER-DEFINED')]


In [55]:
print(type(all_nodes[0][0]))

<class 'str'>
